In [1]:
# Librerías.
import pandas as pd
import Funciones as f
import unidecode

In [ ]:
%%capture
%run "9. Promedios de tiempos de respuesta.ipynb"

In [3]:
# Lista de todas las posibles redes sociales.
Redes_Sociales = [
    'twitter', 
    'facebook', 
    'instagram', 
    'threads',
    'tiktok',
    'youtube',
    'whatsapp',
    'telegram'
]

In [4]:
# Eliminar acentos y convertir a minúsculas en la columna Red_Social.
for Nombre_df, df in dfs_Finales.items():
   df['Red_Social'] = df['Red_Social'].apply(lambda x: unidecode.unidecode(str(x)).lower() if pd.notna(x) else x)

In [5]:
# Crear una columna para cada red social.

for Nombre_df, df in dfs_Finales.items():
   for Red in Redes_Sociales:
       
       # Crear la columna con nombre limpio.
       Nombre_Columna = f"{Red}"
       
       # Verificar si la red social está en el string de cada fila.
       df[Nombre_Columna] = df['Red_Social'].str.contains(Red, case = False, na = False).astype(int)

In [6]:
# Crear diccionario de mapeo para renombrar columnas.
Mapeo_Redes_Sociales = {Red: Red.capitalize() for Red in Redes_Sociales}

# Aplicar el renombrado a todos los DataFrames.
for Nombre_df, df in dfs_Finales.items():
   # Verificar qué columnas existen en este DataFrame.
   Columnas_Existentes = [Columna for Columna in Redes_Sociales if Columna in df.columns]
   
   if Columnas_Existentes:       
       # Crear mapeo solo para las columnas que existen.
       Mapeo_Actual = {Columna: Columna.capitalize() for Columna in Columnas_Existentes}
       
       # Renombrar las columnas.
       df.rename(columns = Mapeo_Actual, inplace=True)

In [ ]:
print("="*80)
print("VERIFICACIÓN: PROCESAMIENTO DE REDES SOCIALES")
print("="*80)

# 1. Verificar existencia de la lista de redes sociales.
print("1. VERIFICACIÓN DE LISTA DE REDES SOCIALES:")
if 'Redes_Sociales' in globals():
    print(f"   ✅ Lista definida con {len(Redes_Sociales)} redes:")
    for i, Red in enumerate(Redes_Sociales, 1):
        print(f"      {i}. {Red}")
else:
    print("   ❌ Lista 'Redes_Sociales' no encontrada")
    # Definir manualmente.
    Redes_Sociales = ['twitter', 'facebook', 'instagram', 'threads', 'tiktok', 'youtube', 'whatsapp', 'telegram']
    print(f"   🔧 Definiendo manualmente: {Redes_Sociales}")

print("\n" + "="*60)

for Nombre_df, df in dfs_Finales.items():
    print(f"\n📱 DATAFRAME: {Nombre_df}")
    print("-" * 50)
    
    # 2. Verificar existencia y estado de la columna Red_Social.
    print("2. VERIFICACIÓN DE COLUMNA RED_SOCIAL:")
    
    if 'Red_Social' in df.columns:
        print("   ✅ Columna 'Red_Social' encontrada")
        
        # Estadísticas básicas.
        Total_Casos = len(df)
        Casos_Validos = df['Red_Social'].notna().sum()
        Casos_Nulos = df['Red_Social'].isna().sum()
        
        print(f"   Total de casos: {Total_Casos}")
        print(f"   Casos válidos: {Casos_Validos} ({(Casos_Validos/Total_Casos)*100:.1f}%)")
        print(f"   Casos nulos: {Casos_Nulos} ({(Casos_Nulos/Total_Casos)*100:.1f}%)")
        
        # Verificar normalización (minúsculas, sin acentos).
        if Casos_Validos > 0:
            Muestra_Valores = df['Red_Social'].dropna().head(10).tolist()
            print(f"   Muestra de valores: {Muestra_Valores[:5]}")
            
            # Verificar que están en minúsculas.
            Valores_Con_Mayusculas = df['Red_Social'].dropna().apply(lambda x: any(c.isupper() for c in str(x))).sum()
            if Valores_Con_Mayusculas == 0:
                print("   ✅ Todos los valores están en minúsculas")
            else:
                print(f"   ⚠️  {Valores_Con_Mayusculas} valores contienen mayúsculas")
    else:
        print("   ❌ Columna 'Red_Social' NO encontrada")
        continue
    
    # 3. Verificar creación de columnas binarias.
    print("\n3. VERIFICACIÓN DE COLUMNAS BINARIAS:")
    
    # Buscar columnas de redes sociales (capitalizadas).
    Redes_Capitalizadas = [Red.capitalize() for Red in Redes_Sociales]
    Columnas_Redes_Encontradas = []
    Columnas_Redes_Faltantes = []
    
    for Red_Cap in Redes_Capitalizadas:
        if Red_Cap in df.columns:
            Columnas_Redes_Encontradas.append(Red_Cap)
        else:
            Columnas_Redes_Faltantes.append(Red_Cap)
    
    print(f"   ✅ Columnas creadas: {len(Columnas_Redes_Encontradas)}/{len(Redes_Capitalizadas)}")
    print(f"   ❌ Columnas faltantes: {len(Columnas_Redes_Faltantes)}")
    
    if Columnas_Redes_Faltantes:
        print(f"      Faltantes: {Columnas_Redes_Faltantes}")
    
    # 4. Verificar valores binarios correctos.
    print("\n4. VERIFICACIÓN DE VALORES BINARIOS:")
    
    for Red_Cap in Columnas_Redes_Encontradas:
        Valores_Unicos = df[Red_Cap].unique()
        Valores_Ordenados = sorted([v for v in Valores_Unicos if pd.notna(v)])
        
        if set(Valores_Ordenados) == {0, 1}:
            print(f"   ✅ {Red_Cap}: Valores binarios correctos [0, 1]")
        elif set(Valores_Ordenados) == {0}:
            print(f"   ⚠️  {Red_Cap}: Solo contiene 0s")
        elif set(Valores_Ordenados) == {1}:
            print(f"   ⚠️  {Red_Cap}: Solo contiene 1s")
        else:
            print(f"   ❌ {Red_Cap}: Valores incorrectos {Valores_Ordenados}")
    
    # 5. Estadísticas de uso por red social.
    print("\n5. ESTADÍSTICAS DE USO POR RED SOCIAL:")
    
    print("   Red Social    | Usuarios | % Uso   | Total")
    print("   " + "-"*45)
    
    for Red_Cap in sorted(Columnas_Redes_Encontradas):
        if Red_Cap in df.columns:
            Usuarios = df[Red_Cap].sum()
            Porcentaje = (Usuarios / Total_Casos) * 100
            print(f"   {Red_Cap:<12} | {Usuarios:8} | {Porcentaje:6.1f}% | {Total_Casos}")
    
    # 6. Análisis de combinaciones de redes.
    print("\n6. ANÁLISIS DE COMBINACIONES:")
    
    if len(Columnas_Redes_Encontradas) > 0:
        # Calcular total de redes por usuario.
        df['Total_Redes_Usadas'] = df[Columnas_Redes_Encontradas].sum(axis=1)
        
        Distribucion_Redes = df['Total_Redes_Usadas'].value_counts().sort_index()
        
        print("   Redes por usuario:")
        for Num_Redes, Cantidad in Distribucion_Redes.items():
            Porcentaje = (Cantidad / Total_Casos) * 100
            print(f"     {Num_Redes} redes: {Cantidad} usuarios ({Porcentaje:.1f}%)")
        
        # Identificar usuarios sin redes.
        Sin_Redes = (df['Total_Redes_Usadas'] == 0).sum()
        if Sin_Redes > 0:
            print(f"   ⚠️  {Sin_Redes} usuarios sin redes sociales detectadas")
    
    # 7. VERIFICACIÓN MANUAL CON CASOS ESPECÍFICOS.
    print("\n7. VERIFICACIÓN MANUAL - CASOS ESPECÍFICOS:")
    
    # Seleccionar 5 casos al azar para verificación manual.
    if len(df) >= 5:
        import random
        random.seed(42)  # Para reproducibilidad.
        Indices_Muestra = random.sample(range(len(df)), min(5, len(df)))
        
        for i, Indice in enumerate(Indices_Muestra, 1):
            Fila = df.iloc[Indice]
            ID_Participante = Fila['ID']
            Red_Social_Original = Fila['Red_Social']
            
            print(f"\n   🔍 CASO {i} - ID: {ID_Participante}")
            print(f"      Red_Social original: '{Red_Social_Original}'")
            
            if pd.notna(Red_Social_Original):
                # Mostrar detección por cada red.
                print("      Detecciones:")
                for Red_Cap in Columnas_Redes_Encontradas:
                    Detectado = Fila[Red_Cap]
                    Red_Minuscula = Red_Cap.lower()
                    
                    # Verificar manualmente si debería estar detectada.
                    Deberia_Detectar = Red_Minuscula in str(Red_Social_Original).lower()
                    
                    Status = "✅" if (Detectado == 1) == Deberia_Detectar else "❌"
                    print(f"        {Red_Cap}: {Detectado} {Status}")
                
                # Total de redes detectadas.
                Total_Detectadas = sum([Fila[Red_Cap] for Red_Cap in Columnas_Redes_Encontradas])
                print(f"      Total redes detectadas: {Total_Detectadas}")
            else:
                print("      Red_Social es NaN")
    
    # 8. Detección de posibles errores o casos edge.
    print("\n8. DETECCIÓN DE CASOS PROBLEMÁTICOS:")
    
    if 'Red_Social' in df.columns:
        # Buscar valores sospechosos en Red_Social.
        Valores_Red_Social = df['Red_Social'].dropna().unique()
        
        # Valores muy largos (podrían ser errores).
        Valores_Largos = [v for v in Valores_Red_Social if len(str(v)) > 100]
        if Valores_Largos:
            print(f"   ⚠️  {len(Valores_Largos)} valores muy largos detectados")
            print(f"      Ejemplo: '{str(Valores_Largos[0])[:50]}...'")
        
        # Valores con caracteres especiales.
        import re
        Valores_Con_Especiales = [v for v in Valores_Red_Social 
                                 if re.search(r'[^\w\s,.-]', str(v))]
        if Valores_Con_Especiales:
            print(f"   ⚠️  {len(Valores_Con_Especiales)} valores con caracteres especiales")
            print(f"      Ejemplos: {Valores_Con_Especiales[:3]}")
        
        # Buscar menciones de redes no incluidas en la lista.
        Otras_Redes_Posibles = ['linkedin', 'snapchat', 'discord', 'reddit', 'twitch']
        for Red in Otras_Redes_Posibles:
            Menciones = df['Red_Social'].str.contains(Red, case=False, na=False).sum()
            if Menciones > 0:
                print(f"   📊 Red no incluida '{Red}': {Menciones} menciones")
    
    # 9. Distribución por categorías políticas.
    print("\n9. USO DE REDES POR CATEGORÍA POLÍTICA:")
    
    if ('Categoria_PASO_2023' in df.columns and len(Columnas_Redes_Encontradas) > 0):
        print("\n   Uso promedio por categoría (% que usa cada red):")
        print("   Categoría           ", end="")
        for Red in Columnas_Redes_Encontradas[:4]:  # Primeras 4 redes.
            print(f"| {Red[:8]:<8}", end="")
        print("| Total")
        print("   " + "-" * (20 + 10 * min(4, len(Columnas_Redes_Encontradas)) + 7))
        
        for Categoria in df['Categoria_PASO_2023'].unique():
            if pd.notna(Categoria):
                df_Cat = df[df['Categoria_PASO_2023'] == Categoria]
                if len(df_Cat) > 0:
                    print(f"   {Categoria[:18]:<18} ", end="")
                    
                    for Red in Columnas_Redes_Encontradas[:4]:
                        Porcentaje = (df_Cat[Red].sum() / len(df_Cat)) * 100
                        print(f"| {Porcentaje:7.1f}%", end="")
                    
                    # Total de redes promedio.
                    Total_Promedio = df_Cat[Columnas_Redes_Encontradas].sum(axis=1).mean()
                    print(f"| {Total_Promedio:5.1f}")

print("\n" + "="*80)
print("✅ VERIFICACIÓN DE REDES SOCIALES COMPLETADA")
print("="*80)

# 10. Resumen ejecutivo.
print("\n" + "="*80)
print("RESUMEN EJECUTIVO")
print("="*80)

for Nombre_df, df in dfs_Finales.items():
    print(f"\n📊 DATAFRAME: {Nombre_df}")
    
    if 'Red_Social' in df.columns and len(Columnas_Redes_Encontradas) > 0:
        # Red más popular.
        Uso_Por_Red = {Red: df[Red].sum() for Red in Columnas_Redes_Encontradas}
        Red_Mas_Popular = max(Uso_Por_Red, key=Uso_Por_Red.get)
        
        # Usuario promedio.
        Redes_Promedio = df[Columnas_Redes_Encontradas].sum(axis=1).mean()
        
        # Cobertura total.
        Usuarios_Con_Al_Menos_Una_Red = (df[Columnas_Redes_Encontradas].sum(axis=1) > 0).sum()
        Cobertura = (Usuarios_Con_Al_Menos_Una_Red / len(df)) * 100
        
        print(f"   🏆 Red más popular: {Red_Mas_Popular} ({Uso_Por_Red[Red_Mas_Popular]} usuarios)")
        print(f"   📊 Redes promedio por usuario: {Redes_Promedio:.1f}")
        print(f"   📈 Cobertura total: {Cobertura:.1f}% tienen al menos una red")
        print(f"   🔧 Columnas binarias creadas: {len(Columnas_Redes_Encontradas)}")

print("\n" + "="*80)
print("✅ ANÁLISIS DE REDES SOCIALES COMPLETADO")
print("="*80)

In [8]:
# Exportar base para el control.
for Nombre_df, df in dfs_Finales.items():
    dfs_Finales[Nombre_df].head(50).to_excel(f'Controles/10. Redes sociales ({Nombre_df}).xlsx', index=False)